$$ ITI \space AI-Pro: \space Intake \space 45 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 2 $$

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir /content/Data
!cp /content/drive/MyDrive/Labs/Lab_2/Data/songsDataset.csv /content/drive/MyDrive/Labs/Lab_2/Data/songs_data.csv /content/Data/


mkdir: cannot create directory ‘/content/Data’: File exists


# `01` Import Necessary Libraries

## `i` Default Libraries

In [5]:
!pip install "numpy<2.0"
!pip install scikit_surprise

In [6]:
import numpy as np
import pandas as pd
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.knns import KNNWithMeans

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [108]:
from surprise.accuracy import rmse

----------------------------

# `02` Load Data

 The dataset will have the following columns :
   - song_id (String) : Unique identified for the song
   - user_id (String) : Unique identifier for the user
   - song_genre (Integer) : An integer representing a genre for the song, value is between 1 and 5, indicating that there are 5 unique genres. Each song can only have 1 genre
   - artist_id (String) : Unique identifier for the author of the song
   - n_listen (Integer) : The number of times this user has heard the song (0 -> 15)
   - publish_year (Integer) : The year of song publishing

In [43]:

data = pd.read_csv("Data/songs_data.csv")
data.head()

,song_id,artist_id,song_genre,user_id,n_listen,publish_year
0,537,368,4,2066,13,2002
1,921,107,1,1179,5,2006
2,352,188,1,1468,11,2013
3,853,370,4,460,9,2020
4,479,408,2,1125,3,2020


--------------------------

# `03` Content-based Filtering

Practice for content-based filtering on dummy data

## `i` Feature Engineering/Selection
Construct the item vector representation matrix from the `data` above

In [70]:
item_vec_repr = pd.pivot_table(data, values='n_listen', columns='song_genre', index='song_id', aggfunc=lambda x: 1 if len(x) > 0 else 0, fill_value=0)

item_vec_repr


song_genre,1,2,3,4,5
song_id,,,,,
1,0,1,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,1,0,0,0
5,0,0,0,0,1
...,...,...,...,...,...
996,1,0,0,0,0
997,1,0,0,0,0
998,0,0,1,0,0


## `ii` Utility Matrix
Construct utility matrix for the loaded dataframe `data`

In [41]:
util_mat = data.pivot(values='n_listen', index='user_id', columns='song_id').fillna(0)

util_mat


song_id,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,15.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,11.0,0.0,6.0
3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,11.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,0.0,9.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
2998,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `iii` Item-Item Similarity Matrix

Construct item-item (Cosine/Adjusted Cosine) similarity matrix.

In [10]:
def adjusted_cosine_sim(vec_a, vec_b):
    """
    Returns the adjusted cosine similarity score between two vectors.

            Parameters:
                vec_a (pandas.Series): Vector A
                vec_b (pandas.Series): Vector B

            Returns:
                sim_score (float): Similarity score between vectors vec_a and vec_b
    """
    avg_vec_a = np.mean(vec_a)
    avg_vec_b = np.mean(vec_b)

    adj_vec_a = vec_a - avg_vec_a
    adj_vec_b = vec_b - avg_vec_b

    dividend = adj_vec_a@adj_vec_b
    divisor = np.linalg.norm(adj_vec_a) * np.linalg.norm(adj_vec_b)

    if divisor ==0:
      return None

    sim_score = dividend/divisor

    return sim_score

In [71]:
sim_mat= np.zeros((item_vec_repr.shape[0], item_vec_repr.shape[0]))

for i in range(item_vec_repr.shape[0]):
  item_u = item_vec_repr.iloc[i].values

  for j in range(i, item_vec_repr.shape[0]):
    if i == j:
      sim_mat[i,j] = 1
    else:
      item_v = item_vec_repr.iloc[j].values
      val = adjusted_cosine_sim(item_u, item_v)
      sim_mat[i,j] = val
      sim_mat[j,i] = val
sim_mat

array([[ 1.  , -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25,  1.  , -0.25, ...,  1.  , -0.25, -0.25],
       [-0.25, -0.25,  1.  , ..., -0.25, -0.25,  1.  ],
       ...,
       [-0.25,  1.  , -0.25, ...,  1.  , -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25,  1.  , -0.25],
       [-0.25, -0.25,  1.  , ..., -0.25, -0.25,  1.  ]])

In [72]:
sim_df = pd.DataFrame(sim_mat, index=item_vec_repr.index, columns=item_vec_repr.index)
sim_df.head()

song_id,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
song_id,,,,,,,,,,,,,,,,,,,,,
1,1.00,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,...,1.00,-0.25,1.00,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25
2,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,...,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,1.00,-0.25,-0.25
3,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,1.00,...,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,1.00
4,1.00,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,...,1.00,-0.25,1.00,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25
5,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,...,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,1.00,-0.25


## `iv` Top-K Candidate Generation

Selet top-K (a k of your choice) similar items for each item (a user of your choice) rated from the similarity matrix above.

In [74]:
user_id =2
listened_songs = util_mat.iloc[user_id][util_mat[user_id] !=0].index
listened_songs

Index([  7,  13,  21,  24,  28,  29,  44,  58,  63,  79,
       ...
       927, 936, 953, 954, 958, 960, 961, 974, 980, 981],
      dtype='int64', name='song_id', length=132)

## `v` Candidate Filtering

Filter out items (your user) has rated from the candidates above.

In [78]:
filtered_sim_df = sim_df.drop(listened_songs, axis =1)
filtered_sim_df

song_id,1,2,3,4,5,6,8,9,10,11,...,991,992,993,994,995,996,997,998,999,1000
song_id,,,,,,,,,,,,,,,,,,,,,
1,1.00,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,...,1.00,-0.25,1.00,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25
2,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,...,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,1.00,-0.25,-0.25
3,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,...,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,1.00
4,1.00,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,...,1.00,-0.25,1.00,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25
5,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,...,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,-0.25,-0.25,1.00,-0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,-0.25,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,1.00,...,-0.25,-0.25,-0.25,-0.25,-0.25,1.00,1.00,-0.25,-0.25,-0.25
997,-0.25,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,1.00,...,-0.25,-0.25,-0.25,-0.25,-0.25,1.00,1.00,-0.25,-0.25,-0.25
998,-0.25,1.00,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,-0.25,...,-0.25,-0.25,-0.25,-0.25,1.00,-0.25,-0.25,1.00,-0.25,-0.25


In [80]:
filtered_sim_df.loc[1].nlargest(k)

,1
song_id,
1,1.0
4,1.0
9,1.0
16,1.0
32,1.0


In [88]:
k=3
d = {}
for item in listened_songs:
  cands = filtered_sim_df.loc[item].nlargest(k)

  for cand in cands.index.to_list():
    if cand in d.keys():
      d[cand][item] = filtered_sim_df[cand][item]
    else:
      d[cand] = {item:filtered_sim_df[cand][item]}

## `vi` Candidate Rating Prediction

Calculate the predicted rating for each of the candidate items.

In [89]:
preds = {}
for key, value in d.items():
  preds[key] = [None]

  num = 0
  denum = 0
  for ref, sim in value.items():
    rating=util_mat[user_id][ref]
    term = rating*sim
    num +=term
    denum += sim
    preds[key].extend([ref, sim, rating])
  pred_rating = num/denum
  preds[key][0] = pred_rating

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
6,8.600000,7,1.0,8.0,156,1.0,6.0,198,1.0,8.0,...,11.0,837.0,1.0,1.0,848.0,1.0,10.0,953.0,1.0,8.0
11,8.600000,7,1.0,8.0,156,1.0,6.0,198,1.0,8.0,...,11.0,837.0,1.0,1.0,848.0,1.0,10.0,953.0,1.0,8.0
14,8.600000,7,1.0,8.0,156,1.0,6.0,198,1.0,8.0,...,11.0,837.0,1.0,1.0,848.0,1.0,10.0,953.0,1.0,8.0
5,7.863636,13,1.0,10.0,21,1.0,10.0,200,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,7.863636,13,1.0,10.0,21,1.0,10.0,200,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,7.863636,13,1.0,10.0,21,1.0,10.0,200,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.214286,24,1.0,6.0,28,1.0,1.0,63,1.0,2.0,...,13.0,980.0,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
8,8.214286,24,1.0,6.0,28,1.0,1.0,63,1.0,2.0,...,13.0,980.0,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
17,8.214286,24,1.0,6.0,28,1.0,1.0,63,1.0,2.0,...,13.0,980.0,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
3,8.137931,29,1.0,2.0,58,1.0,9.0,105,1.0,12.0,...,5.0,909.0,1.0,6.0,927.0,1.0,2.0,NaN,NaN,NaN


In [100]:
highest_len = 0
for key in preds.keys():
  length = len(preds[key])
  if length > highest_len:
    highest_len = length
cols = ['predicted_rating']
for i in range(1, (highest_len+1)//2):
  cols.extend([f'ref_{i}', f'ref{i}_sim'])

pd.DataFrame.from_dict(preds, orient='index',columns=cols)


,predicted_rating,ref_1,ref1_sim,ref_2,ref2_sim,ref_3,ref3_sim,ref_4,ref4_sim,ref_5,...,ref_41,ref41_sim,ref_42,ref42_sim,ref_43,ref43_sim,ref_44,ref44_sim,ref_45,ref45_sim
6,8.600000,7,1.0,8.0,156,1.0,6.0,198,1.0,8.0,...,11.0,837.0,1.0,1.0,848.0,1.0,10.0,953.0,1.0,8.0
11,8.600000,7,1.0,8.0,156,1.0,6.0,198,1.0,8.0,...,11.0,837.0,1.0,1.0,848.0,1.0,10.0,953.0,1.0,8.0
14,8.600000,7,1.0,8.0,156,1.0,6.0,198,1.0,8.0,...,11.0,837.0,1.0,1.0,848.0,1.0,10.0,953.0,1.0,8.0
5,7.863636,13,1.0,10.0,21,1.0,10.0,200,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,7.863636,13,1.0,10.0,21,1.0,10.0,200,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,7.863636,13,1.0,10.0,21,1.0,10.0,200,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.214286,24,1.0,6.0,28,1.0,1.0,63,1.0,2.0,...,13.0,980.0,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
8,8.214286,24,1.0,6.0,28,1.0,1.0,63,1.0,2.0,...,13.0,980.0,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
17,8.214286,24,1.0,6.0,28,1.0,1.0,63,1.0,2.0,...,13.0,980.0,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
3,8.137931,29,1.0,2.0,58,1.0,9.0,105,1.0,12.0,...,5.0,909.0,1.0,6.0,927.0,1.0,2.0,NaN,NaN,NaN


--------------------------

# `04` KNN Item-based Colaborative Filtering

Practice for Using Scikit Surprise Library

## `i` Data Loading

Load `songsDataset.csv` file into a dataframe

In [90]:
df = pd.read_csv('Data/songsDataset.csv')
df.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


## `ii` Prepare Data

Procedures to Follow:
- Instantiate the Reader Object (see, [Documentation](https://surprise.readthedocs.io/en/stable/reader.html))
- Load the Data into `surprise.dataset.Dataset` (see, [Documentation](https://surprise.readthedocs.io/en/stable/dataset.html))
- Build the full (i.e. without folds) `surprise.Trainset` (see, [Documentation](https://surprise.readthedocs.io/en/stable/trainset.html#:~:text=It%20is%20used%20by%20the%20fit()%20method%20of%20every%20prediction%20algorithm.%20You%20should%20not%20try%20to%20build%20such%20an%20object%20on%20your%20own%20but%20rather%20use%20the%20Dataset.folds()%20method%20or%20the%20DatasetAutoFolds.build_full_trainset()%20method.))

In [103]:
reader = Reader()


In [104]:
data = Dataset.load_from_df(df, reader)
data

## `iii` Initialize the `KNNWithMeans` Model

**Note**: `KNNWithMeans` uses the normalized ratings instead of the raw ones. (See [Documentation](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans))

**Hint**: Use $k=10$ and configure `sim_options` to be:
- item_based
- pearson

In [105]:
knn_model = KNNWithMeans(k=10, sim_options={'user_based': False, 'name':'Pearson'})

## `iv` Fit the Model on Data

In [106]:
train, test = train_test_split(data, test_size=0.1, random_state=20)
knn_model.fit(train)
knn_model

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [109]:
preds = knn_model.test(test)
rmse(preds)

RMSE: 1.5619


1.561908136198834

## `v` Calculate Predicted Rating $\hat{r}$ for User $199988$

**Hine**: you can use `.predict()` method of the model (see [Documentaion](https://surprise.readthedocs.io/en/stable/getting_started.html?highlight=.predict#train-on-a-whole-trainset-and-the-predict-method:~:text=pred%20%3D%20algo.predict(uid%2C%20iid%2C%20r_ui%3D4%2C%20verbose%3DTrue)))

In [131]:
uid = 199988
listened_songs = df[df['userID']==uid]['songID'].index
filtered_df = df.drop(listened_songs, axis=0)
cand_songs = filtered_df['songID'].unique()

In [144]:
uid = 199988
iids = df['songID'].unique()

song_predictions = [knn_model.predict(uid, iid).est for iid in cand_songs]
iid_pred = zip(cand_songs, song_predictions)

song_predictions = pd.DataFrame(iid_pred, columns=['songID', 'predicted_rating'])
song_predictions.set_index('songID', inplace=True)
song_predictions.head()

,predicted_rating
songID,
90409,4.750183
91266,4.728662
8063,4.453699
24427,4.518279
105433,4.607436


## `vi` Recommend Top 10 Songs

In [145]:
song_predictions_sorted = song_predictions.sort_values(by='predicted_rating', ascending=False)
song_predictions_sorted.head(10)

,predicted_rating
songID,
92881,5.0
122065,5.0
132189,5.0
2263,5.0
71582,5.0
52611,5.0
40712,5.0
60888,5.0
62954,5.0


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Mahmoud \space Shawqi $$